In [1]:
from rucio.client import Client
import itertools
from tqdm import tqdm

import random
import string

In [2]:
client = Client()
client.whoami()

{'account': 'root',
 'account_type': 'SERVICE',
 'status': 'ACTIVE',
 'deleted_at': None,
 'updated_at': '2024-11-11T18:21:21',
 'email': None,
 'suspended_at': None,
 'created_at': '2024-11-11T18:21:21'}

# Initialize RSE

In [3]:
sites = {
    "T2_US_SDSC": {
        "hostname": "xrootd-sense-ucsd-redirector-111.sdsc.optiputer.net",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//temp"
        # "prefix": "//"
    },
    "T2_US_Caltech": {
        "hostname": "redir-01.t2-sense.ultralight.org",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//"
        # "prefix": "//store/temp"
    },
    "T2_US_Caltech_Test": {
        "hostname": "sense-redir-01.ultralight.org",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//" 
    },
    "T1_US_FNAL": {
        "hostname": "cmssense2-origin-2841-1.fnal.gov",
        "scheme": "davs",
        "port": 2880,
        "prefix": "//dcache/store/temp"
    },
    "T2_US_Nebraska_Test": {
        "hostname": "red-sense-dtn1.unl.edu",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//"
    },
}

In [4]:
for site in sites.values():
    site.update({"impl": "rucio.rse.protocols.gfal.Default",
                "domains": {
                    "lan": {
                        "read": 1,
                        "write": 1,
                        "delete": 1
                    },
                    "wan": {
                        "read": 1,
                        "write": 1,
                        "delete": 1,
                        "third_party_copy_read": 1,
                        "third_party_copy_write": 1
                    }
                }})

for site, site_info in sites.items():
    client.add_rse(site)
    client.add_protocol(site, site_info)
    client.set_local_account_limit("root", site, "-1")
    client.add_rse_attribute(site, "fts", "https://cmsftssrv3.fnal.gov:8446")
    # client.add_rse_attribute(site, "fts", "https://fts3-pilot.cern.ch:8446")
    client.add_rse_attribute(site, "lfn2pfn_algorithm", "cmstfc")
    client.add_rse_attribute(site, "verify_checksum", "false")

iterator = itertools.combinations(sites.keys(), 2)
for src, dst in iterator:
    client.add_distance(src, dst, {"ranking": 1, "distance": 1})
    client.add_distance(dst, src, {"ranking": 1, "distance": 1})

client.add_scope("root", "cms")

True

In [5]:
print(f"RSEs: {[i['rse'] for i in client.list_rses()]}")

RSEs: ['T2_US_SDSC', 'T2_US_Caltech', 'T2_US_Caltech_Test', 'T1_US_FNAL', 'T2_US_Nebraska_Test']


# Add Rule

In [6]:
PRIORITY = 5

# 65000-105000 at SDSC, 170000-180000 at Caltech, 105000-115000 at Caltech_Test, 50000-63000 at FNAL

# DATASET = "65000,74000" # SOURCE SDSC
# DATASET = "170000,180000" # SOURCE Caltech
DATASET = "50000,59000" # SOURCE FNAL

NUM_FILES_PER_DATASET = 1000

# SOURCE = "T2_US_Caltech"
# SOURCE = "T2_US_SDSC"
SOURCE = "T1_US_FNAL"

DESTINATION = "T2_US_SDSC"
# DESTINATION = "T2_US_Caltech"

In [7]:
container_suffix = ''.join(random.choices(string.ascii_uppercase, k=4))
dataset_start, dataset_end = int(DATASET.split(",")[0]), int(DATASET.split(",")[1])+1000

print("Adding files to datasets")
datasets_for_containers = []
for dataset in tqdm(range(dataset_start, dataset_end, 1000)):
    files = [f"/store/data/Run2018A/EGamma/MINIAOD/UL2018_MiniAODv2-v1/{dataset}/testSourceFile{j}.root" for j in range(NUM_FILES_PER_DATASET) if (j % 2 == 0) or (j % 4 == 1)]
    dids = [{"scope": "cms", "name": f} for f in files]
    dids_w_bytes = [{"scope": "cms", "name": f, "bytes": 4294967296} for f in files]
    client.add_replicas(SOURCE, dids_w_bytes)
    client.add_dataset("cms", f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}")
    client.add_files_to_datasets([{"scope": "cms", "name": f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}", "dids": dids}])

    datasets_for_containers.append({"scope": "cms", "name": f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}"})

random.shuffle(datasets_for_containers)

print("Adding datasets to container")
client.add_container("cms", f"/SenseTest/Run3/{container_suffix}")
client.add_datasets_to_containers([{"scope": "cms", "name": f"/SenseTest/Run3/{container_suffix}", "dids": datasets_for_containers}])

Adding files to datasets


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:38<00:00,  3.89s/it]


Adding datasets to container


True

In [8]:
print("Adding replication rule")
rule = client.add_replication_rule(dids=[{"scope": "cms", "name": f"/SenseTest/Run3/{container_suffix}"}],
    copies=1,
    meta="sense",
    rse_expression=DESTINATION,
    priority=PRIORITY,
    source_replica_expression=SOURCE)

print("Rule ID:", rule[0])

Adding replication rule


Rule ID: a805108b15954cbd99dadd8eea37c780


In [9]:
[i for i in client.list_account_rules("root")]

[{'account': 'root',
  'copies': 1,
  'activity': 'User Subscriptions',
  'priority': 5,
  'updated_at': datetime.datetime(2024, 11, 11, 20, 3, 35),
  'scope': 'cms',
  'expires_at': None,
  'grouping': 'DATASET',
  'name': '/SenseTest/Run3/WUBP',
  'weight': None,
  'notification': 'NO',
  'comments': None,
  'did_type': 'CONTAINER',
  'locked': False,
  'stuck_at': None,
  'child_rule_id': None,
  'state': 'REPLICATING',
  'locks_ok_cnt': 0,
  'purge_replicas': False,
  'eol_at': None,
  'id': 'a805108b15954cbd99dadd8eea37c780',
  'error': None,
  'locks_replicating_cnt': 7500,
  'ignore_availability': False,
  'split_container': False,
  'rse_expression': 'T2_US_SDSC',
  'locks_stuck_cnt': 0,
  'ignore_account_limit': False,
  'meta': '"sense"',
  'subscription_id': None,
  'source_replica_expression': 'T1_US_FNAL',
  'created_at': datetime.datetime(2024, 11, 11, 20, 3),
  'bytes': None}]